In [1]:
# Hydro-Québec Complete Energy Independence Analysis
# Solar + Battery + V2H Analysis with Rate Flex D Optimization
# Using YOUR actual heating data from Hydro-Québec API
#
# Requirements:
#     pip install hydroq-api pandas matplotlib python-dotenv numpy scipy

import warnings
warnings.filterwarnings('ignore')

from hydroq_api import HydroQuebec
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import os
from dotenv import load_dotenv
from scipy import stats

In [2]:
load_dotenv()

USERNAME = os.getenv('HYDRO_QUEBEC_USERNAME')
PASSWORD = os.getenv('HYDRO_QUEBEC_PASSWORD')

if not USERNAME or not PASSWORD:
    raise ValueError("Credentials not found! Create .env file with your credentials")

print("="*80)
print(" HYDRO-QUÉBEC COMPLETE ENERGY INDEPENDENCE ANALYSIS")
print(" Solar + Battery + V2H | Rate Flex D | Critical Loads with YOUR Heating Data")
print("="*80)

 HYDRO-QUÉBEC COMPLETE ENERGY INDEPENDENCE ANALYSIS
 Solar + Battery + V2H | Rate Flex D | Critical Loads with YOUR Heating Data


In [3]:



# ## 1. CONFIGURATION

RATE_FLEX_D = {
    'system_access_charge_per_day': 0.49,
    'tier1_price_summer': 0.0628,
    'tier2_price_summer': 0.0968,
    'tier1_price_winter_normal': 0.0428,
    'tier2_price_winter_normal': 0.0736,
    'peak_event_price': 0.4508,
    'peak_events_per_winter': 30,
    'peak_hours_per_winter': 120,
}

RATE_D = {
    'system_access_charge_per_day': 0.4978,
    'tier1_price': 0.0673,
    'tier2_price': 0.1038,
}

BATTERY_CONFIG = {
    'depth_of_discharge': 0.90,
    'inverter_efficiency': 0.95,
    'battery_degradation_10yr': 0.80,
    'safety_margin': 1.10,
    'round_trip_efficiency': 0.90,
}

SOLAR_CONFIG = {
    'annual_production_per_kw': 1200,
    'winter_production_factor': 0.35,
    'summer_production_factor': 0.65,
    'system_efficiency': 0.85,
    'degradation_per_year': 0.005,
}

SOLAR_COSTS = {
    'cost_per_watt': 2.90,
    'hq_incentive_per_kw': 1000,
    'max_incentive_percent': 0.40,
    'installation_cost': 0,
    'inverter_upgrade_cost': 2000,
}

# Mid-size SUV EV options with V2H capability
V2H_SUVS = {
    'Volkswagen ID.4 (82 kWh)': {
        'battery_capacity_kwh': 82,
        'usable_capacity_kwh': 77,
        'v2h_equipment_cost': 8000,  # HagerEnergy system
        'vehicle_cost_cad': 55000,
        'installation_cost': 3000,
        'v2h_available_canada': 'Coming 2025-2026',
        'notes': 'Requires ID Software 3.5+, HagerEnergy compatible system'
    },
    'Hyundai Ioniq 5 (84 kWh)': {
        'battery_capacity_kwh': 84,
        'usable_capacity_kwh': 77,
        'v2h_equipment_cost': 7500,  # Wallbox Quasar 2 or similar
        'vehicle_cost_cad': 58000,
        'installation_cost': 3000,
        'v2h_available_canada': 'Coming soon (V2L currently available)',
        'notes': 'Currently V2L only, V2H announced for "near term"'
    },
    'Kia EV6 (77.4 kWh)': {
        'battery_capacity_kwh': 77.4,
        'usable_capacity_kwh': 70,
        'v2h_equipment_cost': 7500,
        'vehicle_cost_cad': 56000,
        'installation_cost': 3000,
        'v2h_available_canada': 'Coming soon (V2L currently available)',
        'notes': 'V2H announced for "near term", same platform as Ioniq 5'
    },
    'Kia EV9 (99.8 kWh)': {
        'battery_capacity_kwh': 99.8,
        'usable_capacity_kwh': 90,
        'v2h_equipment_cost': 7500,
        'vehicle_cost_cad': 75000,
        'installation_cost': 3000,
        'v2h_available_canada': 'Available now (limited)',
        'notes': '3-row SUV, V2H available with Wallbox Quasar 2'
    },
}

STATIC_BATTERIES = {
    'Tesla Powerwall 3': {'capacity_kwh': 13.5, 'cost_cad': 16500, 'warranty_years': 10},
    'LG Chem RESU 16': {'capacity_kwh': 16.0, 'cost_cad': 12000, 'warranty_years': 10},
    'Enphase IQ Battery 5P': {'capacity_kwh': 5.0, 'cost_cad': 7500, 'warranty_years': 15},
}

# Critical loads - we'll calculate heating from actual data!
CRITICAL_LOADS_BASE = {
    'Refrigerator': {'watts': 150, 'hours_per_day': 24, 'essential': True},
    'Freezer': {'watts': 100, 'hours_per_day': 24, 'essential': True},
    'Internet/Router': {'watts': 20, 'hours_per_day': 24, 'essential': True},
    'LED Lighting': {'watts': 100, 'hours_per_day': 6, 'essential': True},
    'Sump Pump': {'watts': 800, 'hours_per_day': 1, 'essential': True},
    'TV/Entertainment': {'watts': 200, 'hours_per_day': 4, 'essential': False},
    'Computer/Office': {'watts': 150, 'hours_per_day': 8, 'essential': False},
}

print("\n✓ Configuration loaded")



✓ Configuration loaded


In [4]:


# ## 2. RETRIEVE AND ANALYZE YOUR CONSUMPTION DATA

print("\nLogging in to Hydro-Québec...")
client = HydroQuebec(USERNAME, PASSWORD)
client.login()
print("✓ Successfully logged in")

end_date = datetime.now()
start_date = end_date - timedelta(days=365)

print(f"\nFetching YOUR consumption data from {start_date.date()} to {end_date.date()}...")
daily_data = client.get_daily_usage(
    start_date=start_date.strftime('%Y-%m-%d'),
    end_date=end_date.strftime('%Y-%m-%d')
)

if not daily_data or not daily_data.get('success'):
    raise Exception("Could not retrieve data")

results = daily_data.get('results', [])
consumption_list = []
for entry in results:
    current = entry.get('courant', {})
    consumption_list.append({
        'date': current.get('dateJourConso'),
        'consumption_kwh': current.get('consoTotalQuot', 0),
        'avg_temperature': current.get('tempMoyenneQuot'),
    })

df = pd.DataFrame(consumption_list)
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').set_index('date')
df = df.dropna(subset=['avg_temperature'])

print(f"✓ Retrieved {len(df)} days of YOUR consumption data")



Logging in to Hydro-Québec...
✓ Successfully logged in

Fetching YOUR consumption data from 2025-01-02 to 2026-01-02...
✓ Retrieved 365 days of YOUR consumption data


In [5]:

# ## 3. ESTIMATE YOUR HEATING CONSUMPTION FROM ACTUAL DATA

print("\n" + "="*80)
print("ANALYZING YOUR ACTUAL HEATING CONSUMPTION")
print("="*80)

# Estimate baseline from mild temperatures
mild_temp_mask = (df['avg_temperature'] >= 15) & (df['avg_temperature'] <= 20)
baseline_consumption = df.loc[mild_temp_mask, 'consumption_kwh'].median()

print(f"\nYour baseline consumption (appliances, hot water, lights): {baseline_consumption:.2f} kWh/day")
print(f"Based on {mild_temp_mask.sum()} days with temperatures 15-20°C")

# Estimate heating (when temp < 18°C)
heating_threshold = 18
df['heating_estimate'] = 0.0
heating_mask = df['avg_temperature'] < heating_threshold

if heating_mask.sum() > 0:
    heating_data = df[heating_mask].copy()
    z_scores = np.abs(stats.zscore(heating_data['consumption_kwh']))
    heating_data_clean = heating_data[z_scores < 3]
    
    if len(heating_data_clean) > 10:
        slope, intercept, r_value, p_value, std_err = stats.linregress(
            heating_data_clean['avg_temperature'], 
            heating_data_clean['consumption_kwh']
        )
        df.loc[heating_mask, 'heating_estimate'] = np.maximum(
            0,
            df.loc[heating_mask, 'consumption_kwh'] - baseline_consumption
        )
        print(f"Heating model correlation (R²): {r_value**2:.3f}")

df['other_estimate'] = np.maximum(
    0,
    df['consumption_kwh'] - df['heating_estimate']
)

# Calculate average heating consumption for winter days
winter_months = [12, 1, 2, 3]
winter_heating = df[df.index.month.isin(winter_months)]['heating_estimate']
avg_winter_heating_kwh = winter_heating.mean()
avg_total_winter_day = df[df.index.month.isin(winter_months)]['consumption_kwh'].mean()

print(f"\nYOUR WINTER HEATING DATA:")
print(f"  Average winter heating: {avg_winter_heating_kwh:.2f} kWh/day")
print(f"  Heating as % of winter consumption: {(avg_winter_heating_kwh/avg_total_winter_day)*100:.1f}%")
print(f"  Total winter daily average: {avg_total_winter_day:.2f} kWh/day")



ANALYZING YOUR ACTUAL HEATING CONSUMPTION

Your baseline consumption (appliances, hot water, lights): 25.70 kWh/day
Based on 77 days with temperatures 15-20°C
Heating model correlation (R²): 0.925

YOUR WINTER HEATING DATA:
  Average winter heating: 142.53 kWh/day
  Heating as % of winter consumption: 84.7%
  Total winter daily average: 168.23 kWh/day


In [6]:

# ## 4. CRITICAL LOADS ANALYSIS WITH YOUR ACTUAL HEATING

print("\n" + "="*80)
print("CRITICAL LOADS ANALYSIS (Using YOUR Heating Data)")
print("="*80)

# Calculate base loads (without heating)
base_essential_kwh = sum(
    (load['watts'] * load['hours_per_day'] / 1000) 
    for load in CRITICAL_LOADS_BASE.values() if load['essential']
)

# Add your actual heating estimate
# For critical loads, we use average winter heating (worst case scenario)
critical_daily_kwh = base_essential_kwh + avg_winter_heating_kwh

# Calculate with all loads
all_loads_kwh = sum(
    load['watts'] * load['hours_per_day'] / 1000 
    for load in CRITICAL_LOADS_BASE.values()
) + avg_winter_heating_kwh

print(f"\nBase essential loads (no heating): {base_essential_kwh:.2f} kWh/day")
print(f"YOUR heating (winter average): +{avg_winter_heating_kwh:.2f} kWh/day")
print(f"Total critical loads (winter): {critical_daily_kwh:.2f} kWh/day")
print(f"All loads (winter): {all_loads_kwh:.2f} kWh/day")

print("\n" + "-"*80)
print(f"{'Load':<30} {'Power (W)':<12} {'Hours/Day':<12} {'Daily kWh':<12} {'Essential'}")
print("-"*80)
for name, specs in CRITICAL_LOADS_BASE.items():
    daily = specs['watts'] * specs['hours_per_day'] / 1000
    essential = "✓" if specs['essential'] else ""
    print(f"{name:<30} {specs['watts']:<12} {specs['hours_per_day']:<12} {daily:<12.2f} {essential}")
print(f"{'YOUR Heating (winter avg)':<30} {'Variable':<12} {'Variable':<12} {avg_winter_heating_kwh:<12.2f} ✓")
print("-"*80)
print(f"{'TOTAL CRITICAL (Winter)':<30} {'':<12} {'':<12} {critical_daily_kwh:<12.2f}")

# Required battery for critical loads
critical_battery_required = (
    critical_daily_kwh * BATTERY_CONFIG['safety_margin'] / 
    (BATTERY_CONFIG['depth_of_discharge'] * BATTERY_CONFIG['inverter_efficiency'])
)

print(f"\n→ Battery needed for 1-day critical loads backup: {critical_battery_required:.1f} kWh")

# Summer critical loads (no heating)
summer_critical_kwh = base_essential_kwh
summer_critical_battery = (
    summer_critical_kwh * BATTERY_CONFIG['safety_margin'] /
    (BATTERY_CONFIG['depth_of_discharge'] * BATTERY_CONFIG['inverter_efficiency'])
)
print(f"→ Battery needed for 1-day summer critical loads: {summer_critical_battery:.1f} kWh")



CRITICAL LOADS ANALYSIS (Using YOUR Heating Data)

Base essential loads (no heating): 7.88 kWh/day
YOUR heating (winter average): +142.53 kWh/day
Total critical loads (winter): 150.41 kWh/day
All loads (winter): 152.41 kWh/day

--------------------------------------------------------------------------------
Load                           Power (W)    Hours/Day    Daily kWh    Essential
--------------------------------------------------------------------------------
Refrigerator                   150          24           3.60         ✓
Freezer                        100          24           2.40         ✓
Internet/Router                20           24           0.48         ✓
LED Lighting                   100          6            0.60         ✓
Sump Pump                      800          1            0.80         ✓
TV/Entertainment               200          4            0.80         
Computer/Office                150          8            1.20         
YOUR Heating (winter avg)  

In [7]:

# ## 5. FULL HOME BATTERY SIZING

df['required_capacity_kwh'] = (
    df['consumption_kwh'] * BATTERY_CONFIG['safety_margin'] / 
    (BATTERY_CONFIG['depth_of_discharge'] * BATTERY_CONFIG['inverter_efficiency'])
)

stats_dict = {
    'Mean': df['consumption_kwh'].mean(),
    'Median': df['consumption_kwh'].median(),
    '95th Percentile': df['consumption_kwh'].quantile(0.95),
    'Maximum': df['consumption_kwh'].max(),
}

required_dict = {
    'Mean': df['required_capacity_kwh'].mean(),
    'Median': df['required_capacity_kwh'].median(),
    '95th Percentile': df['required_capacity_kwh'].quantile(0.95),
    'Maximum': df['required_capacity_kwh'].max(),
}

print("\n" + "="*80)
print("FULL HOME BATTERY SIZING")
print("="*80)
print(f"\n{'Scenario':<20} {'Daily Consumption':<20} {'Battery Required':<20} {'Coverage'}")
print("-"*80)
for key in ['Mean', 'Median', '95th Percentile', 'Maximum']:
    coverage = '~50% of days' if key in ['Mean', 'Median'] else ('95% of days' if key == '95th Percentile' else '100% of days')
    print(f"{key:<20} {stats_dict[key]:<20.1f} {required_dict[key]:<20.1f} {coverage}")



FULL HOME BATTERY SIZING

Scenario             Daily Consumption    Battery Required     Coverage
--------------------------------------------------------------------------------
Mean                 88.7                 114.2                ~50% of days
Median               67.0                 86.2                 ~50% of days
95th Percentile      204.1                262.6                95% of days
Maximum              242.6                312.1                100% of days


In [8]:

# ## 6. SOLAR SYSTEM SIZING

annual_consumption = df['consumption_kwh'].sum()
avg_daily_consumption = df['consumption_kwh'].mean()

solar_size_kw = annual_consumption / (
    SOLAR_CONFIG['annual_production_per_kw'] * SOLAR_CONFIG['system_efficiency']
)

annual_solar_production = solar_size_kw * SOLAR_CONFIG['annual_production_per_kw'] * SOLAR_CONFIG['system_efficiency']

print("\n" + "="*80)
print("SOLAR SYSTEM SIZING")
print("="*80)
print(f"\nYour annual consumption: {annual_consumption:.0f} kWh")
print(f"Recommended solar system: {solar_size_kw:.1f} kW")
print(f"Expected annual production: {annual_solar_production:.0f} kWh")
print(f"Coverage: {(annual_solar_production/annual_consumption)*100:.1f}% of annual needs")

gross_solar_cost = solar_size_kw * 1000 * SOLAR_COSTS['cost_per_watt']
hq_incentive = min(
    solar_size_kw * SOLAR_COSTS['hq_incentive_per_kw'],
    gross_solar_cost * SOLAR_COSTS['max_incentive_percent']
)
net_solar_cost = gross_solar_cost - hq_incentive

print(f"\n→ Solar system cost: ${gross_solar_cost:,.0f}")
print(f"→ Hydro-Québec incentive (NEW 2025!): -${hq_incentive:,.0f}")
print(f"→ Net solar cost: ${net_solar_cost:,.0f}")



SOLAR SYSTEM SIZING

Your annual consumption: 32386 kWh
Recommended solar system: 31.8 kW
Expected annual production: 32386 kWh
Coverage: 100.0% of annual needs

→ Solar system cost: $92,076
→ Hydro-Québec incentive (NEW 2025!): -$31,750
→ Net solar cost: $60,326


In [9]:

# ## 7. RATE FLEX D & ROI

tier1_kwh = min(40, avg_daily_consumption) * 365
tier2_kwh = max(0, annual_consumption - tier1_kwh)
annual_cost_rate_d = (
    RATE_D['system_access_charge_per_day'] * 365 +
    tier1_kwh * RATE_D['tier1_price'] +
    tier2_kwh * RATE_D['tier2_price']
)

peak_hours_total = RATE_FLEX_D['peak_hours_per_winter']
peak_kwh_per_hour = 4
peak_kwh_avoided = peak_hours_total * peak_kwh_per_hour * 0.5

winter_months = [12, 1, 2, 3]
winter_daily_avg = df[df.index.month.isin(winter_months)]['consumption_kwh'].mean()
summer_daily_avg = df[~df.index.month.isin(winter_months)]['consumption_kwh'].mean()

winter_tier1 = min(40, winter_daily_avg) * 30 * 4
winter_tier2 = max(0, winter_daily_avg * 120 - winter_tier1)
summer_tier1 = min(40, summer_daily_avg) * 30 * 8
summer_tier2 = max(0, summer_daily_avg * 240 - summer_tier1)

annual_cost_flex_d = (
    RATE_FLEX_D['system_access_charge_per_day'] * 365 +
    winter_tier1 * RATE_FLEX_D['tier1_price_winter_normal'] +
    winter_tier2 * RATE_FLEX_D['tier2_price_winter_normal'] +
    summer_tier1 * RATE_FLEX_D['tier1_price_summer'] +
    summer_tier2 * RATE_FLEX_D['tier2_price_summer'] +
    (peak_hours_total * peak_kwh_per_hour - peak_kwh_avoided) * RATE_FLEX_D['peak_event_price']
)

consumption_after_solar = annual_consumption * 0.20
cost_with_solar_flex_d = (
    RATE_FLEX_D['system_access_charge_per_day'] * 365 +
    consumption_after_solar * RATE_FLEX_D['tier1_price_winter_normal'] * 0.4 +
    consumption_after_solar * RATE_FLEX_D['tier1_price_summer'] * 0.6
)

annual_savings = annual_cost_flex_d - cost_with_solar_flex_d

print("\n" + "="*80)
print("RATE COMPARISON & ROI")
print("="*80)
print(f"\nYour current cost (Rate D): ${annual_cost_rate_d:,.0f}/year")
print(f"With Rate Flex D (no solar): ${annual_cost_flex_d:,.0f}/year")
print(f"With Solar + Rate Flex D: ${cost_with_solar_flex_d:,.0f}/year")
print(f"\nAnnual savings: ${annual_savings:,.0f}")

solar_payback = net_solar_cost / annual_savings
print(f"\n→ Solar payback: {solar_payback:.1f} years")


RATE COMPARISON & ROI

Your current cost (Rate D): $3,010/year
With Rate Flex D (no solar): $2,444/year
With Solar + Rate Flex D: $534/year

Annual savings: $1,910

→ Solar payback: 31.6 years


In [10]:

# ## 8. SOLUTION 1: SOLAR + STATIC BATTERY (Critical Loads)

print("\n" + "="*80)
print("SOLUTION 1: SOLAR + STATIC BATTERY (Critical Loads - Winter)")
print("="*80)

best_critical = None
best_critical_cost = float('inf')

for name, specs in STATIC_BATTERIES.items():
    units = np.ceil(critical_battery_required / specs['capacity_kwh'])
    total_cap = units * specs['capacity_kwh']
    total_cost = units * specs['cost_cad'] + 3000
    
    if total_cost < best_critical_cost:
        best_critical_cost = total_cost
        best_critical = {'name': name, 'units': int(units), 'capacity': total_cap, 'cost': total_cost}
    
    print(f"\n{name}:")
    print(f"  Units: {int(units)} × {specs['capacity_kwh']} kWh = {total_cap:.1f} kWh")
    print(f"  Cost: ${total_cost:,.0f}")

total_cost_critical = net_solar_cost + best_critical_cost
print(f"\n→ TOTAL (Solar + Battery): ${total_cost_critical:,.0f}")
print(f"→ Covers: {critical_battery_required/critical_daily_kwh:.1f} days of critical loads in winter")



SOLUTION 1: SOLAR + STATIC BATTERY (Critical Loads - Winter)

Tesla Powerwall 3:
  Units: 15 × 13.5 kWh = 202.5 kWh
  Cost: $250,500

LG Chem RESU 16:
  Units: 13 × 16.0 kWh = 208.0 kWh
  Cost: $159,000

Enphase IQ Battery 5P:
  Units: 39 × 5.0 kWh = 195.0 kWh
  Cost: $295,500

→ TOTAL (Solar + Battery): $219,326
→ Covers: 1.3 days of critical loads in winter


In [11]:

# ## 9. SOLUTION 2: SOLAR + STATIC BATTERY (Full Home)

print("\n" + "="*80)
print("SOLUTION 2: SOLAR + STATIC BATTERY (Full Home - 95%)")
print("="*80)

full_required = required_dict['95th Percentile']
best_full = None
best_full_cost = float('inf')

for name, specs in STATIC_BATTERIES.items():
    units = np.ceil(full_required / specs['capacity_kwh'])
    total_cap = units * specs['capacity_kwh']
    total_cost = units * specs['cost_cad'] + 3000
    
    if total_cost < best_full_cost:
        best_full_cost = total_cost
        best_full = {'name': name, 'units': int(units), 'capacity': total_cap, 'cost': total_cost}
    
    print(f"\n{name}:")
    print(f"  Units: {int(units)} × {specs['capacity_kwh']} kWh = {total_cap:.1f} kWh")
    print(f"  Cost: ${total_cost:,.0f}")

total_cost_full = net_solar_cost + best_full_cost
print(f"\n→ TOTAL (Solar + Battery): ${total_cost_full:,.0f}")


SOLUTION 2: SOLAR + STATIC BATTERY (Full Home - 95%)

Tesla Powerwall 3:
  Units: 20 × 13.5 kWh = 270.0 kWh
  Cost: $333,000

LG Chem RESU 16:
  Units: 17 × 16.0 kWh = 272.0 kWh
  Cost: $207,000

Enphase IQ Battery 5P:
  Units: 53 × 5.0 kWh = 265.0 kWh
  Cost: $400,500

→ TOTAL (Solar + Battery): $267,326


In [12]:



# ## 10. SOLUTION 3: SOLAR + V2H (Mid-Size SUV)

print("\n" + "="*80)
print("SOLUTION 3: SOLAR + V2H (Mid-Size SUV EVs)")
print("="*80)

print(f"\n{'Vehicle':<30} {'Battery':<10} {'V2H Cost':<12} {'Days Backup':<15} {'Total System':<15} {'Status'}")
print("-"*80)

v2h_solutions = []
for vehicle, specs in V2H_SUVS.items():
    v2h_cost = specs['v2h_equipment_cost'] + specs['installation_cost']
    system_cost = net_solar_cost + v2h_cost + SOLAR_COSTS['inverter_upgrade_cost']
    
    usable = specs['usable_capacity_kwh'] * 0.80  # 80% for home, 20% for driving
    days_critical = usable / critical_daily_kwh
    days_full = usable / avg_daily_consumption
    
    v2h_solutions.append({
        'vehicle': vehicle,
        'battery': specs['battery_capacity_kwh'],
        'usable': usable,
        'v2h_cost': v2h_cost,
        'system_cost': system_cost,
        'days_critical': days_critical,
        'days_full': days_full,
        'vehicle_cost': specs['vehicle_cost_cad'],
        'status': specs['v2h_available_canada']
    })
    
    print(f"{vehicle:<30} {specs['battery_capacity_kwh']:<10.0f} ${v2h_cost:<11,} {days_full:<15.1f} ${system_cost:<14,} {specs['v2h_available_canada']}")

print("\n⚠️  V2H NOTES:")
print("  • Vehicle cost NOT included above")
print("  • VW ID.4 & Hyundai/Kia: V2H coming to Canada 2025-2026")
print("  • Kia EV9: V2H available now (limited installations)")
print("  • 80% battery for home, 20% reserved for driving")
print("  • Requires CSA-certified bidirectional charger")


SOLUTION 3: SOLAR + V2H (Mid-Size SUV EVs)

Vehicle                        Battery    V2H Cost     Days Backup     Total System    Status
--------------------------------------------------------------------------------
Volkswagen ID.4 (82 kWh)       82         $11,000      0.7             $73,325.931372549 Coming 2025-2026
Hyundai Ioniq 5 (84 kWh)       84         $10,500      0.7             $72,825.931372549 Coming soon (V2L currently available)
Kia EV6 (77.4 kWh)             77         $10,500      0.6             $72,825.931372549 Coming soon (V2L currently available)
Kia EV9 (99.8 kWh)             100        $10,500      0.8             $72,825.931372549 Available now (limited)

⚠️  V2H NOTES:
  • Vehicle cost NOT included above
  • VW ID.4 & Hyundai/Kia: V2H coming to Canada 2025-2026
  • Kia EV9: V2H available now (limited installations)
  • 80% battery for home, 20% reserved for driving
  • Requires CSA-certified bidirectional charger


In [13]:


# ## 11. FINAL RECOMMENDATIONS

print("\n" + "="*80)
print("PERSONALIZED RECOMMENDATIONS (Based on YOUR Data)")
print("="*80)

print(f"""
📊 YOUR ENERGY PROFILE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Annual consumption: {annual_consumption:,.0f} kWh
Average daily: {avg_daily_consumption:.1f} kWh
Winter heating: {avg_winter_heating_kwh:.1f} kWh/day ({(avg_winter_heating_kwh/avg_total_winter_day)*100:.0f}% of winter use)
Critical loads (winter): {critical_daily_kwh:.1f} kWh/day
Current annual cost: ${annual_cost_rate_d:,.0f}

OPTION 1: CRITICAL LOADS ONLY (Recommended for Quebec)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
• Solar: {solar_size_kw:.1f} kW (${net_solar_cost:,.0f} after rebate)
• Battery: {best_critical['name']} ({best_critical['capacity']:.1f} kWh)
• Total Cost: ${total_cost_critical:,.0f}
• Backup: {critical_battery_required/critical_daily_kwh:.1f} days critical (winter), {critical_battery_required/summer_critical_kwh:.1f} days (summer)
• Bill Reduction: ~80% (${annual_savings:,.0f}/year)
• Payback: ~{solar_payback:.0f} years (solar only)
✓ Keeps fridge, freezer, heating, internet running
✓ Most cost-effective for Quebec's reliable grid

OPTION 2: FULL HOME BACKUP
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
• Solar: {solar_size_kw:.1f} kW
• Battery: {best_full['name']} ({best_full['capacity']:.1f} kWh)
• Total Cost: ${total_cost_full:,.0f}
• Backup: 1 full day (95% of days)
• Bill Reduction: ~80%
• Payback: ~{(total_cost_full/annual_savings):.0f} years
✓ Complete peace of mind
✓ For rural areas or critical equipment

OPTION 3: V2H WITH MID-SIZE SUV (Future-Ready)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Best options:
1. VW ID.4: ${v2h_solutions[0]['system_cost']:,.0f} + ${v2h_solutions[0]['vehicle_cost']:,} = ${v2h_solutions[0]['system_cost'] + v2h_solutions[0]['vehicle_cost']:,.0f}
   • {v2h_solutions[0]['battery']:.0f} kWh battery → {v2h_solutions[0]['days_full']:.1f} days backup
   
2. Hyundai Ioniq 5: ${v2h_solutions[1]['system_cost']:,.0f} + ${v2h_solutions[1]['vehicle_cost']:,} = ${v2h_solutions[1]['system_cost'] + v2h_solutions[1]['vehicle_cost']:,.0f}
   • {v2h_solutions[1]['battery']:.0f} kWh battery → {v2h_solutions[1]['days_full']:.1f} days backup

✓ Dual purpose: transportation + home backup
✓ Massive battery capacity ({v2h_solutions[0]['days_critical']:.1f} days of critical loads!)
✓ V2H coming to Canada 2025-2026
⚠️  Initial investment includes vehicle

🎯 MY RECOMMENDATION FOR YOU:

Based on YOUR heating data ({avg_winter_heating_kwh:.1f} kWh/day):

→ START WITH OPTION 1 unless you:
  • Live in rural area with frequent outages
  • Have medical equipment needing 24/7 power
  • Already own/plan to buy V2H-capable SUV

YOUR heating is {(avg_winter_heating_kwh/avg_total_winter_day)*100:.0f}% of winter consumption!
A {critical_battery_required:.1f} kWh battery handles critical loads + heating for {critical_battery_required/critical_daily_kwh:.1f} days.

🇨🇦 QUEBEC ADVANTAGE:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
• Most reliable grid in North America
• NEW $1,000/kW solar rebate (saves ${hq_incentive:,.0f}!)
• Rate Flex D: Save by avoiding peak events
• Low rates make payback longer but system still worthwhile
• Solar production: ~{SOLAR_CONFIG['annual_production_per_kw']} kWh/kW/year in southern Quebec
""")

print("="*80)
print("Analysis complete! Next steps: Get quotes from local installers.")
print("="*80)

# ## 12. EXPORT RESULTS

output_file = f"personalized_energy_analysis_{datetime.now().strftime('%Y%m%d')}.csv"
summary = pd.DataFrame([{
    'Your Annual Consumption (kWh)': annual_consumption,
    'Your Avg Daily (kWh)': avg_daily_consumption,
    'Your Winter Heating (kWh/day)': avg_winter_heating_kwh,
    'Your Critical Loads Winter (kWh/day)': critical_daily_kwh,
    'Solar Size (kW)': solar_size_kw,
    'Solar Net Cost': net_solar_cost,
    'Critical Battery (kWh)': critical_battery_required,
    'Full Battery (kWh)': full_required,
    'Option 1 Total Cost': total_cost_critical,
    'Option 2 Total Cost': total_cost_full,
    'Annual Savings': annual_savings,
    'Solar Payback (years)': solar_payback,
}])
summary.to_csv(output_file, index=False)
print(f"\n✓ Your personalized analysis saved to: {output_file}")


PERSONALIZED RECOMMENDATIONS (Based on YOUR Data)

📊 YOUR ENERGY PROFILE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Annual consumption: 32,386 kWh
Average daily: 88.7 kWh
Winter heating: 142.5 kWh/day (85% of winter use)
Critical loads (winter): 150.4 kWh/day
Current annual cost: $3,010

OPTION 1: CRITICAL LOADS ONLY (Recommended for Quebec)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
• Solar: 31.8 kW ($60,326 after rebate)
• Battery: LG Chem RESU 16 (208.0 kWh)
• Total Cost: $219,326
• Backup: 1.3 days critical (winter), 24.6 days (summer)
• Bill Reduction: ~80% ($1,910/year)
• Payback: ~32 years (solar only)
✓ Keeps fridge, freezer, heating, internet running
✓ Most cost-effective for Quebec's reliable grid

OPTION 2: FULL HOME BACKUP
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
• Solar: 31.8 kW
• Battery: LG Chem RESU 16 (272.0 kWh)
• Total Cost: $267,326
• Backup: 1 full day (95% of days)
• Bill Reduction: ~80%
• Payback: ~140 years
✓ Complete peace